# Clustering Algorithm Methodology and Testing
Will Wright

### Purpose and Context

I've worked with several unsupervised clustering algorithms as part of my professional and academic career, but I had never built one from scratch.  Because the data for this project is binary and I had never worked exclusively with this type of data, I decided to test my creativity and see if I could build a functioning algorithmm from scratch with no outside influence.  

Once complete, this algorithm will be used to classify the resource data from the Slay the Spire character datasets into resource-personas (i.e. the unique 'builds' possible to win with each character on Ascension 20).

In [356]:
# Load packages
import shutil
from os import listdir
import json
import glob
import os
import numpy as np
import pandas as pd
import random
import copy
from heapq import nsmallest

# increase viewable dataframe rows and columns
pd.set_option('display.max_rows', 50)
pd.set_option('display.max_columns', 20)

# set random seed
random.seed(30)

___
## Clustering Algorithm

After much whiteboarding, testing, and deep thought on the matter, I devised the following algorithm for clustering:

**Clustering Algorithm**
1. Start with clusters equal to the number of nodes (k=m)
2. While k>=1:
    1. Calculate the distance between each node and every other node where distance is the sum of resources that don't match (i.e. if a card is in both decks, that resource adds 0 distance, but it if is in one and not the other, then it adds 1 distance.  "Distance" is the sum of all these resource differences)
    2. For each node, calculate the nearest node(s) via distance + a user-defined tolerance distance and merge with the node via taking the mean 
    3. For each merged node, calculate the distances to all other merged nodes and the min and max distances per merged node.
    4. For each min and max distance, add a weighted user-defined tolerance distance percent where the weight is relative to k such that weight = k/m (this applies a 100% weight when k=m and decreases to 0% as k decreases. i.e. it takes larger 'steps' toward a lower k and slows down as k approaches 0)
    5. Drop the node(s) for which the min distance is smaller than or equal to the min distance + tolerance and, of those, the max distance is the smaller than the max distance + tolerance (this steps adds maximum distance between the nodes)
    6. Store the resulting centroids, k, and average distance between nodes
    7. Set k equal to the resulting number of undropped nodes
4. Review average distance for k=2 through k=m
5. Select a reasonable k based on the average distance between nodes

### Simulating Test Data

In order to know if the algorithm works, we'll want to see how it performs on a dataset where we know results.

Lets imagine we have 20 resources in 100 games and 4 somewhat-tight clusters.  In order to do this, we'll create 4 medoids by drawing from the binomial distribution with p = 0.2, 0.4, 0.6, and 0.8 for each draw being a 1 instead of a 0 for each resource within a medoid.

In [2]:
first_medoid = np.random.binomial(1, 0.2, 20)
second_medoid = np.random.binomial(1, 0.4, 20)
third_medoid = np.random.binomial(1, 0.6, 20)
fourth_medoid = np.random.binomial(1, 0.8, 20)

print(first_medoid)
print(second_medoid)
print(third_medoid)
print(fourth_medoid)


[1 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 1]
[0 0 0 1 0 1 1 0 0 0 1 1 0 0 0 0 0 1 0 0]
[0 1 0 1 1 1 0 0 1 0 0 1 1 1 0 1 0 0 1 0]
[1 1 1 1 1 1 1 0 1 0 1 1 1 0 1 1 1 1 1 1]


We'll want a function to measure the distance between games (as described in step 2 of the algorithm):

In [3]:
def cluster_distance_calculator(cluster1_input, cluster2_input):
    '''
    input: arrays of binary data for two clusters
    output: a distance measurement
    method: distance is the sum of differences in the binary data, by position
    '''
    distance = sum(abs(cluster1_input-cluster2_input))
    return(distance)

In [4]:
# test it out for the distance between the first medoid and the others
print(cluster_distance_calculator(first_medoid, first_medoid))
print(cluster_distance_calculator(first_medoid, second_medoid))
print(cluster_distance_calculator(first_medoid, third_medoid))
print(cluster_distance_calculator(first_medoid, fourth_medoid))


0
9
11
16


As expected, there is 0 distance for the first medoid compared to itself. Both the second and the third medoids have 9 total differences and the fourth medoid has 14. We'll want to see what the confusion matrix of each cluster's distance from each other cluster in a function for easy and comprehensive evaluation:

In [5]:
def cluster_confusioner(cluster_list_input):
    '''
    input: a list of clusters of equal length
    output: a matrix which applies the cluster_distance_calculator to each pair of clusters
    '''
    distance_matrix = np.empty((len(cluster_list_input), len(cluster_list_input)))
    
    # iterate through each comparison to populate the matrix
    for i in range(len(cluster_list_input)):
        for j in range(len(cluster_list_input)):
            distance_matrix[i,j] = cluster_distance_calculator(cluster_list_input[i], cluster_list_input[j])
    
    return(distance_matrix)
    

In [6]:
cluster_confusioner([first_medoid, second_medoid, third_medoid, fourth_medoid])

array([[ 0.,  9., 11., 16.],
       [ 9.,  0., 10., 11.],
       [11., 10.,  0.,  9.],
       [16., 11.,  9.,  0.]])

Here, we can see those same 0, 9, 9, 14 values across the first row and down the first column as well as the distances between the other clusters.  It looks like the maximum distance is 14 (difference between the first and fourth medoid) and the minimum distance is a three-way tie between medoids 1:2, 1:3, and 2:4 with a distance of 9.

Next, we'll want to create 24 similar games per medoid to simulate a situation in which there were 4 winning sets of resources.  We'll do this by randomly selecting 0-25% of the elements in each cluster and flipping them. such that we'll have 75% to 100% similarity between each game intended for a cluster and its medoid (I say _intended_ because it's possible that, after applying the random changes, it becomes more similar to a different medoid).

In [7]:
def cluster_creator(medoid_input, difference_percent_range, n_games):
    '''
    input: medoid_input is a one-dimensional array of binary data
           difference_percent_range is a list with a min and max percent (e.g. [0,0.25] for 0-25%); cannot do <1% 
           n_games is the number of games needed in the output
    output: a list of n games with the speficied similarity to the medoid_input
    '''
    
    simulated_games = []
    
    for i in range(n_games):
        # select how many elements will be changed
        # must multiply by 100 and add 1 due to randrange needing integers and being exclusive with the high end
        percent_change = random.randrange(difference_percent_range[0]*100, (difference_percent_range[1]+0.01)*100, 1)/100
        
        # convert the percent to an integer by multiplying by the total number of elements and rounding
        element_change = round(len(medoid_input)*percent_change)
        
        # select which elements will be changed
        element_change_positions = []
        for j in range(element_change):
            element_change_positions.append(random.randrange(0,len(medoid_input)))
        
        # change those elements
        simulated_game = copy.copy(medoid_input)
        for k in range(len(element_change_positions)):
            if simulated_game[element_change_positions[k]]==1:
                simulated_game[element_change_positions[k]]=0
            else:
                simulated_game[element_change_positions[k]]=1
        
        # append to list of games
        simulated_games.append(simulated_game)
    
    return(simulated_games)
        

In [8]:
# create the cluster and add the medoid to it
first_medoid_cluster = cluster_creator(first_medoid, [0,0.25], 24)

In [9]:
first_medoid_cluster

[array([0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0]),
 array([1, 1, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 1]),
 array([1, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1]),
 array([0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 1]),
 array([1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1]),
 array([1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1]),
 array([1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0]),
 array([1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 0, 0, 0, 1, 0, 1]),
 array([1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1]),
 array([0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1]),
 array([1, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1]),
 array([1, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0]),
 array([1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1]),
 array([1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 1]),
 array([1, 0, 0, 0, 

Looks right, but just to test, let's make sure the differences between each simulated game and the medoid is less than or equal to 25% (5 elements of the 20):

In [10]:
distances = []
for i in range(len(first_medoid_cluster)):
    distances = distances + [cluster_distance_calculator(first_medoid, first_medoid_cluster[i])]
print(max(distances))

4


Perfect! now to create the other clusters of games around the other medoids.

In [11]:
second_medoid_cluster = cluster_creator(second_medoid, [0,0.25], 24)
third_medoid_cluster = cluster_creator(third_medoid, [0,0.25], 24)
fourth_medoid_cluster = cluster_creator(fourth_medoid, [0,0.25], 24)

Finally, we can stitch all the games together into one dataframe:

In [12]:
simulated_games = [first_medoid] + first_medoid_cluster + [second_medoid] + second_medoid_cluster +\
[third_medoid] + third_medoid_cluster + [fourth_medoid] + fourth_medoid_cluster

In [13]:
simulated_games = pd.DataFrame(np.array(simulated_games))

In [14]:
simulated_games

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19
0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,1
1,0,0,0,0,0,0,0,0,0,1,0,0,0,1,0,0,0,0,0,0
2,1,1,0,0,0,0,1,0,1,0,0,0,1,1,0,0,0,0,0,1
3,1,0,1,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,1
4,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,1,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,1,1,1,1,1,0,1,1,1,0,1,0,1,0,1,1,0,1,1,1
96,0,1,1,1,1,1,1,0,0,0,1,1,1,0,1,1,1,1,1,1
97,1,1,1,1,1,1,1,0,1,0,1,1,1,0,1,1,1,1,1,1
98,0,1,0,1,0,1,1,0,1,0,1,1,1,0,1,1,0,1,1,1


### Building the Clustering Algorithm
So now, in total, we have 100 games of 20 resources with 25 in each cluster centered around 4 medoids. This is roughly what we should expect to find with the actual game data so it should serve as a relevant proxy.

To restate the goal of the Algorithm:  
**Clustering Algorithm**
1. Start with clusters equal to the number of nodes (k=m)
2. While k>=1:
    1. Calculate the distance between each node and every other node where distance is the sum of resources that don't match (i.e. if a card is in both decks, that resource adds 0 distance, but it if is in one and not the other, then it adds 1 distance.  "Distance" is the sum of all these resource differences)
    2. For each node, calculate the nearest node(s) via distance + a user-defined tolerance distance and merge with the node via taking the mean 
    3. For each merged node, calculate the distances to all other merged nodes and the min and max distances per merged node.
    4. For each min and max distance, add a weighted user-defined tolerance distance percent where the weight is relative to k such that weight = k/m (this applies a 100% weight when k=m and decreases to 0% as k decreases. i.e. it takes larger 'steps' toward a lower k and slows down as k approaches 0)
    5. Drop the node(s) for which the min distance is smaller than or equal to the min distance + tolerance and, of those, the max distance is the smaller than the max distance + tolerance (this steps adds maximum distance between the nodes)
    6. Store the resulting centroids, k, and average distance between nodes
    7. Set k equal to the resulting number of undropped nodes
4. Review average distance for k=2 through k=m
5. Select a reasonable k based on the average distance between nodes

As per 2A., we'll want to be able to calculate the distance between each node and every other node:

In [15]:
def node_distancer(resource_dataframe_input):
    '''
    input: resouce_dataframe_input is a dataframe with resources in the columns and games in the rows with 
            binary data filling the table.
           k_input is the number of clusters
    output: k centroids
    '''
    # create empty array to hold all the distances comparing each combination
    all_node_distances = np.zeros([len(resource_dataframe_input),len(resource_dataframe_input)])
    # calculate distance between the ith game and the jth game
    for i in range(len(all_node_distances)):
        for j in range(len(all_node_distances)):
            all_node_distances[i,j] = cluster_distance_calculator(resource_dataframe_input.iloc[i], resource_dataframe_input.iloc[j])
    
    return(all_node_distances)

In [16]:
simulated_node_distances = node_distancer(simulated_games)

In [17]:
simulated_node_distances

array([[ 0.,  3.,  4., ..., 16., 14., 16.],
       [ 3.,  0.,  7., ..., 19., 15., 17.],
       [ 4.,  7.,  0., ..., 12., 10., 12.],
       ...,
       [16., 19., 12., ...,  0.,  4.,  2.],
       [14., 15., 10., ...,  4.,  0.,  6.],
       [16., 17., 12., ...,  2.,  6.,  0.]])

Above is a 100x100 array which shows the distance of each node to every other node.  Below is just the first row, which compares the first node to all 100 other nodes, separated by cluster:

In [18]:
print(simulated_node_distances[0,0:25])
print(simulated_node_distances[0,25:50])
print(simulated_node_distances[0,50:75])
print(simulated_node_distances[0,75:100])

[0. 3. 4. 2. 3. 0. 0. 1. 3. 0. 2. 3. 4. 1. 2. 0. 1. 4. 1. 3. 2. 0. 2. 1.
 2.]
[ 9.  9. 10.  7. 11. 11. 11.  8.  5.  9. 11.  9.  9.  8. 10. 11.  6.  8.
  9. 11.  8.  9.  8. 10. 10.]
[11.  9. 12. 10.  8. 12.  9. 10. 14.  9. 10. 11. 13. 10. 13. 11.  8. 10.
 12. 10. 12.  7. 11. 11. 12.]
[16. 15. 17. 16. 16. 16. 12. 15. 15. 14. 14. 16. 15. 13. 14. 16. 16. 16.
 14. 12. 14. 16. 16. 14. 16.]


Above is the first row of what I'm calling a 'node' distance separated into each of the medoid-centered groupings.  In this case, the node is 1 game, but we'll eventually be merging games of resources together so it's easier to call these the more generic 'node' than calling it 'the combination of game 1, game 10,..., game n'.  

What it's showing is that the distance between the first node and itself is, as expected, 0 and the distances between the first node and the others centered around that medoid ranges between 0 and 5.  Compared to the second medoid and the simulated games around it, we see values from 9 to 12. We see a difference of 8-13 for the third medoid cluster and 12-17 for the fourth medoid cluster. In all, it seems to be working as expected.

At step 2B. in the algorithm, we calculate the nearest node and combine the two by taking the average of the resources. As can be seen, the nearest of all the nodes (that isn't itself) is 1 and there are 7 nodes that have the same distance.  This means that 7 nodes have only a 1-resource difference to the first node. I'm not exactly sure how to handle ties, but it seems reasonable to simply take an average across all nodes that are equally close.  I can imagine a better (and more complex) strategy doing backtracking and seeing which node-pairing results in a smaller distance in the n+1 iteration and going with that, but I'll keep this version simple. 

As per this plan, let's create a function which creates a new node based on an average of the equally-close nodes:

In [19]:
def node_averager(list_of_nodes):
    return(np.mean((list_of_nodes), axis = 0))

Next, for each row of 100 distances, we'll want to select the node(s) with the equally-minimal distance from the original node to send to the `node_averager()` and store the resulting nodes along with indices for the nodes included.  In other words, in the case of the first iteration when the nodes are equal to the games, we're finding the most similar other game(s) and creating an average new node.

In [20]:
def new_noder(node_table_input, node_distance_table_input, node_index, tolerance_input):
    '''
    inputs: resource_table_input: a table with games in the rows and resources in the columns (mxn)
            node_distance_table_input: an array of node distances (mxm)
            node_index: an integer value (should range from 0 to m)
            tolerance_input: value to be added to the minimum distance to be included in the closest nodes
    output: a tuple of a new average node and a list of the nodes averaged together
    '''
    node_distances = node_distance_table_input[node_index]
    min_distance = min(node_distances[1:])+tolerance_input # find the closest node(s) that aren't the primary node
    closest_node_indices = np.where(node_distances<=min_distance)[0] # [0] since these are 1-dimensional slices
    
    # grab the closest nodes and put into a list
    closest_nodes = []
    for i in range(len(closest_node_indices)):
        closest_nodes.append(node_table_input.iloc[closest_node_indices[i]].values)
    
    # grab primary_node and add to the list of closest_nodes
    primary_node = node_table_input.iloc[node_index].values
    closest_nodes.append(primary_node)
    
    # take an average of the primary and closest node(s)
    new_node = node_averager(closest_nodes)
    
    return([new_node,[node_index,closest_node_indices]])
    

In [21]:
new_node = new_noder(simulated_games, simulated_node_distances, 0,0)

Excellent! We now have a way of generating a new node, which is the average of the closest nodes (within some tolerance), along with the indices for the primary node and the nodes closest.  

As per the algorithm, we'll want to do this for all nodes before calculating the distances between each of them and removing the closest.

In [311]:
def node_updater(node_table_input, node_distance_table_input, tolerance_input = 1):
    '''
    input: node_table_intput: a dataframe with observations in rows and features in columns with values between 0 and 1
           node_distance_table_input: an array of distances between nodes
           tolerance_input: value to be added to the minimum distance to be included in the closest nodes
    output: a dataframe of the hybrid average-closest nodes
    '''
    new_resource_list = []
    for i in range(len(node_table_input)):
        new_resource_list.append(new_noder(node_table_input, node_distance_table_input,i,tolerance_input)[0])
    
    # convert to dataframe
    updated_node_table = pd.DataFrame(np.array(new_resource_list))
    
    return(updated_node_table)

In [23]:
simulated_games_step2 = node_updater(simulated_games, simulated_node_distances,1)

In [24]:
simulated_games_step2

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19
0,1.000000,0.000000,0.000000,0.0,0.083333,0.0,0.000000,0.083333,0.000000,0.083333,0.0,0.0,0.000000,1.0,0.0,0.0,0.000000,0.0,0.083333,0.916667
1,0.000000,0.000000,0.000000,0.0,0.000000,0.0,0.000000,0.000000,0.000000,1.000000,0.0,0.0,0.000000,1.0,0.0,0.0,0.000000,0.0,0.000000,0.000000
2,1.000000,1.000000,0.000000,0.0,0.000000,0.0,1.000000,0.000000,1.000000,0.000000,0.0,0.0,1.000000,1.0,0.0,0.0,0.000000,0.0,0.000000,1.000000
3,1.000000,0.000000,0.666667,0.0,1.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.000000,1.0,0.0,0.0,0.000000,0.0,0.000000,1.000000
4,0.000000,0.000000,0.000000,0.0,0.000000,0.0,0.000000,1.000000,0.000000,0.000000,0.0,0.0,0.000000,1.0,0.0,0.0,0.666667,0.0,0.000000,1.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,1.000000,1.000000,1.000000,1.0,1.000000,0.0,1.000000,1.000000,1.000000,0.000000,1.0,0.0,1.000000,0.0,1.0,1.0,0.000000,1.0,1.000000,1.000000
96,0.000000,1.000000,1.000000,1.0,1.000000,1.0,1.000000,0.000000,0.333333,0.000000,1.0,1.0,1.000000,0.0,1.0,1.0,1.000000,1.0,1.000000,1.000000
97,0.909091,0.909091,1.000000,1.0,1.000000,1.0,0.909091,0.000000,1.000000,0.000000,1.0,1.0,0.909091,0.0,1.0,1.0,1.000000,1.0,1.000000,1.000000
98,0.000000,1.000000,0.000000,1.0,0.000000,1.0,1.000000,0.000000,1.000000,0.000000,1.0,1.0,1.000000,0.0,1.0,1.0,0.000000,1.0,1.000000,1.000000


With the hybrid-nodes at this iteration complete, we're on to step 2C: For each merged node, calculate the distances to all other merged nodes and the min and max distances per merged node. We can start by simply using the node_distancer function on the hybrid nodes in 'simulated_games_step2'.

In [25]:
node_distance_table_step2 = node_distancer(simulated_games_step2)

In [26]:
i = 0

In [27]:
# taking a look at the distances between the first hybrid node and every other hybrid node:
node_distance_table_step2[i]

array([ 0.        ,  3.08333333,  4.41666667,  1.91666667,  2.91666667,
        0.        ,  0.        ,  0.5       ,  3.41666667,  0.        ,
        2.25      ,  3.08333333,  4.25      ,  0.69444444,  2.41666667,
        0.        ,  0.69444444,  4.41666667,  0.5       ,  3.41666667,
        2.41666667,  0.        ,  2.75      ,  0.5       ,  2.41666667,
        9.41666667,  9.41666667,  9.53571429,  7.41666667, 10.91666667,
       11.08333333, 10.75      ,  8.08333333,  5.25      ,  9.41666667,
       11.25      ,  9.41666667,  9.25      ,  8.41666667,  9.88333333,
       10.91666667,  6.08333333,  8.08333333,  9.41666667, 10.58333333,
        8.25      ,  9.41666667,  8.25      , 10.08333333,  9.36111111,
       10.58333333,  9.25      , 11.91666667,  9.91666667,  7.91666667,
       11.41666667,  9.08333333, 10.25      , 13.75      ,  9.08333333,
       10.25      , 10.91666667, 12.91666667, 10.41666667, 12.58333333,
       10.91666667,  7.75      ,  9.91666667, 11.91666667, 10.25

Next, we calculate the min and max distances:

In [28]:
min(node_distance_table_step2[i][[s for s in list(range(len(node_distance_table_step2[i]))) if s != i]])

0.0

In [29]:
max(node_distance_table_step2[i])

16.416666666666664

It looks like the 0th node has a min distance of 1.73 (excluding the distance to itself) and a max distance of 15.87.  What we want to know is which nodes share the minimum distance, be it 1.73 or something smaller, then, of those, remove the node with the smallest maximum distance.  In order to do this, we'll create a function which returns those min and max values.

In [30]:
def node_distance_min_maxer(node_distance_table_input):
    '''
    input: node_distance_table_input: mxm array of distances between nodes
    output: list of lists containing the minimum and maximum distances per node
    '''
    node_min_maxs= []
    for i in range(len(node_distance_table_input)):
        node_min = min(node_distance_table_input[i][[s for s in list(range(len(node_distance_table_input[i]))) \
                                                     if s != i]])
        node_max = max(node_distance_table_input[i])
        node_min_maxs.append([node_min, node_max])
    return(node_min_maxs)

In [31]:
node_distance_min_max_step2 = node_distance_min_maxer(node_distance_table_step2)

In [357]:
# preview of min/max distances
node_distance_min_max_step2[0:10]

[[0.0, 16.416666666666664],
 [2.75, 18.75],
 [4.0, 13.666666666666668],
 [1.2222222222222223, 15.666666666666666],
 [0.6666666666666666, 17.333333333333336],
 [0.0, 16.416666666666664],
 [0.0, 16.416666666666664],
 [0.49999999999999994, 16.583333333333332],
 [3.25, 15.0],
 [0.0, 16.416666666666664]]

Great, now we need a way to subset to which node has the min first value and the minimum-max (minmax) second:

In [33]:
def extract_nth(list_input, n): 
    return [element[n] for element in list_input] 

In [280]:
min_distance = min(extract_nth(node_distance_min_max_step2,0))

In [281]:
min_distance

0.0

2D. After testing with larger datasets, it became apparent that I'll want to establish a distance_tolerance_pct, which is a percent to add to the min and minmax distances in an effort to capture more nodes to be removed.  This is important because the computation is pretty intense and when we're dealing with thousands of observations/features where we're expecting a relatively small number of clusters, we aren't concerned with the clusters created for the large n or large m so the faster we can reduce while not losing the integrity of the reduction, the better.

In [282]:
distance_tolerance_pct = 0 # testing with 0, then we'll increase to see the effect

In [283]:
min_distance = min_distance * (1 + distance_tolerance_pct)

In [284]:
min_distance

0.0

In [285]:
# using '<=' instead of '==' since we're adding the tolerance
node_min_indices = np.where(extract_nth(node_distance_min_max_step2,0)<=min_distance)[0]

In [286]:
node_min_indices

array([ 0,  5,  6,  9, 15, 21, 25, 34, 36, 43, 46, 75, 80, 86, 90, 91, 97])

In [287]:
min_distance_node_distances = [node_distance_min_max_step2[i] for i in node_min_indices]

In [288]:
# preview of the min and max distances for which the node shares the minimum distance to the other nodes
min_distance_node_distances

[[0.0, 16.416666666666664],
 [0.0, 16.416666666666664],
 [0.0, 16.416666666666664],
 [0.0, 16.416666666666664],
 [0.0, 16.416666666666664],
 [0.0, 16.416666666666664],
 [0.0, 12.88888888888889],
 [0.0, 12.88888888888889],
 [0.0, 12.88888888888889],
 [0.0, 12.88888888888889],
 [0.0, 12.88888888888889],
 [0.0, 18.636363636363633],
 [0.0, 18.636363636363633],
 [0.0, 18.636363636363633],
 [0.0, 18.636363636363633],
 [0.0, 18.636363636363633],
 [0.0, 18.636363636363633]]

In [39]:
minmax_distance = min(extract_nth(min_distance_node_distances,1))

In [289]:
# the minimum-max distance between nodes which share the minimum distance
minmax_distance

12.88888888888889

As with the min distance, we'll add the tolerance pct to the upper bound as well (0 in the case of this first test):

In [290]:
minmax_distance = minmax_distance * (1 + distance_tolerance_pct)

In [291]:
minmax_distance

12.88888888888889

Finally, we have the min and minmax distance conditions for which to subset:

In [292]:
node_removal_distances = np.asarray([min_distance, minmax_distance])

In [293]:
node_removal_distances

array([ 0.        , 12.88888889])

We can see that the 0, 1, 2, 3, 4, and 5th index of the min node distances are all tied at [0, 13].  This being the case, we should remove those nodes from simulated_games_step2 and repeat the process.  What this means is we'll start with k=100 clusters, then go to k=95 since we're removing 6 nodes.  In other words, this is an algorithm which will return arbitrary viable values for k and this makes sense since we can imagine data which has distances that share min and max properties.

In [294]:
node_removal_indices = np.where((extract_nth(node_distance_min_max_step2,0)<=node_removal_distances[0]) &\
                                (extract_nth(node_distance_min_max_step2,1)<=node_removal_distances[1]))[0]

In [295]:
# node indices to be removed
node_removal_indices.tolist()

[25, 34, 36, 43, 46]

In [296]:
# nodes to be removed share the following values for min and minmax distances:
[node_distance_min_max_step2[i] for i in node_removal_indices.tolist()]

[[0.0, 12.88888888888889],
 [0.0, 12.88888888888889],
 [0.0, 12.88888888888889],
 [0.0, 12.88888888888889],
 [0.0, 12.88888888888889]]

As per 2E., we'll want to drop those nodes:

In [297]:
simulated_games_step3 = simulated_games_step2.drop(node_removal_indices)

In [298]:
simulated_games_step3

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19
0,1.000000,0.000000,0.000000,0.0,0.083333,0.0,0.000000,0.083333,0.000000,0.083333,0.0,0.0,0.000000,1.0,0.0,0.0,0.000000,0.0,0.083333,0.916667
1,0.000000,0.000000,0.000000,0.0,0.000000,0.0,0.000000,0.000000,0.000000,1.000000,0.0,0.0,0.000000,1.0,0.0,0.0,0.000000,0.0,0.000000,0.000000
2,1.000000,1.000000,0.000000,0.0,0.000000,0.0,1.000000,0.000000,1.000000,0.000000,0.0,0.0,1.000000,1.0,0.0,0.0,0.000000,0.0,0.000000,1.000000
3,1.000000,0.000000,0.666667,0.0,1.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.000000,1.0,0.0,0.0,0.000000,0.0,0.000000,1.000000
4,0.000000,0.000000,0.000000,0.0,0.000000,0.0,0.000000,1.000000,0.000000,0.000000,0.0,0.0,0.000000,1.0,0.0,0.0,0.666667,0.0,0.000000,1.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,1.000000,1.000000,1.000000,1.0,1.000000,0.0,1.000000,1.000000,1.000000,0.000000,1.0,0.0,1.000000,0.0,1.0,1.0,0.000000,1.0,1.000000,1.000000
96,0.000000,1.000000,1.000000,1.0,1.000000,1.0,1.000000,0.000000,0.333333,0.000000,1.0,1.0,1.000000,0.0,1.0,1.0,1.000000,1.0,1.000000,1.000000
97,0.909091,0.909091,1.000000,1.0,1.000000,1.0,0.909091,0.000000,1.000000,0.000000,1.0,1.0,0.909091,0.0,1.0,1.0,1.000000,1.0,1.000000,1.000000
98,0.000000,1.000000,0.000000,1.0,0.000000,1.0,1.000000,0.000000,1.000000,0.000000,1.0,1.0,1.000000,0.0,1.0,1.0,0.000000,1.0,1.000000,1.000000


Looks like this process works well so we just need to convert it to a function and ensure we get the same results

In [270]:
def drop_closest_nodes(node_table_input, node_minmax_distances, distance_tolerance_pct = 0):
    '''
    input: node_table_input: table with observations in rows, features in columns, and binary values
           mode_minmax_distance: list of lists containing the min and max values to every other node
           distance_tolerance: for larger datasets, it may be reasonable to drop more than just the nearest nodes
               such that k is reduced more quickly in the beginning. This value is the percent to be added to the 
               min/max values used to calculate which nodes to drop
    output: reduced_node_table: equal to node_table_input without the closest nodes
    '''
    # calculate the min distance of the first element in each of the min/max lists
    min_distance = min(extract_nth(node_minmax_distances,0)) + distance_tolerance_pct
    
    # add the distance_tolerance_pct to the min_distance
    min_distance = min_distance * (1 + distance_tolerance_pct)
    
    # calculate indices of the nodes which contain the minimum
    node_min_indices = np.where(extract_nth(node_minmax_distances,0)<=min_distance)[0]
    
    # generate a list of min/max distances for the nodes which contain the min distance
    min_distance_node_distances = [node_minmax_distances[i] for i in node_min_indices]
    
    # of those distances, calculate the minimum max distance (the second element in the distance lists)
    minmax_distance = min(extract_nth(min_distance_node_distances,1))
    
    # add the distance_tolerance_pct to the minmax_distance
    minmax_distance = minmax_distance * (1 + distance_tolerance_pct)
    
    # create an array containing the mininum and minimum maximum distance
    node_removal_distances = np.asarray([min_distance, minmax_distance])
    
    # calculate the node indices from the distance table which are less than or equal to the node_removal_distances
    node_removal_indices = np.where((extract_nth(node_minmax_distances,0)<=node_removal_distances[0]) &\
                                (extract_nth(node_minmax_distances,1)<=node_removal_distances[1]))[0]
    
    # drop the specified nodes from the resource table
    reduced_node_table = node_table_input.drop(node_removal_indices)
    
    return(reduced_node_table)

In [299]:
drop_closest_nodes(simulated_games_step2, node_distance_min_max_step2,0)==simulated_games_step3

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19
0,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True
1,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True
2,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True
3,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True
4,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True
96,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True
97,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True
98,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True


#### Testing impact of adjusting the distance tolerance parameter
Looks like they're the same! And now we can test the impact of adjusting the tolerance on how much the dataframe is reduced.  We can see that with a tolerance of 0, 5 rows are removed.  As we increase this figure, we should expect more and more rows to be removed.

In [305]:
drop_closest_nodes(simulated_games_step2, node_distance_min_max_step2,0.1).shape

(95, 20)

No impact with a 10% tolerance, but lets see how it looks as we increase up to 100%:

In [301]:
drop_closest_nodes(simulated_games_step2, node_distance_min_max_step2,0.2).shape

(95, 20)

In [302]:
drop_closest_nodes(simulated_games_step2, node_distance_min_max_step2,0.3).shape

(87, 20)

In [303]:
drop_closest_nodes(simulated_games_step2, node_distance_min_max_step2,0.4).shape

(80, 20)

In [304]:
drop_closest_nodes(simulated_games_step2, node_distance_min_max_step2,0.5).shape

(62, 20)

In [306]:
drop_closest_nodes(simulated_games_step2, node_distance_min_max_step2,0.6).shape

(60, 20)

In [307]:
drop_closest_nodes(simulated_games_step2, node_distance_min_max_step2,0.7).shape

(60, 20)

In [308]:
drop_closest_nodes(simulated_games_step2, node_distance_min_max_step2,0.8).shape

(49, 20)

In [309]:
drop_closest_nodes(simulated_games_step2, node_distance_min_max_step2,0.9).shape

(49, 20)

In [310]:
drop_closest_nodes(simulated_games_step2, node_distance_min_max_step2,1.0).shape

(37, 20)

Perfect! So now we have a tuning parameter we can use to speed up the convergence.

At this point, we have everything working and just need to stick all the pieces together:

In [312]:
def node_reducer(node_dataframe_input, tolerance_input, distance_tolerance_pct):
    '''
    input: node_dataframe_input: a dataframe with nodes in the rows and binary features in the columns
           tolerance_input: value to be added to the minimum distance to be included in the closest nodes
           distance_tolerance_pct: for larger datasets, it may be reasonable to drop more than just the nearest nodes
               such that k is reduced more quickly in the beginning. This value is the percent to be added to the 
               min/max values used to calculate which nodes to drop
    output: a reduced version of the input table which creates hybrid nodes and removes the node(s) for which there is
        the least difference to the other nodes.
    '''
    
    # calculate node distances
    node_distances = node_distancer(node_dataframe_input)
    
    # create new hybrid nodes
    new_nodes = node_updater(node_dataframe_input, node_distances, tolerance_input)
    
    # calculate hybrid node distances
    new_node_distances = node_distancer(new_nodes)
    
    # calculate the min and max distances per new hybrid node
    new_node_minmax_distances = node_distance_min_maxer(new_node_distances)
    
    # drop the closest node(s)
    reduced_node_dataframe = drop_closest_nodes(new_nodes, new_node_minmax_distances, distance_tolerance_pct)
    
    return(reduced_node_dataframe)


In [50]:
node_reducer(simulated_games, 1)

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19
0,1.000000,0.000000,0.000000,0.0,0.083333,0.0,0.000000,0.083333,0.000000,0.083333,0.0,0.0,0.000000,1.0,0.0,0.0,0.000000,0.0,0.083333,0.916667
1,0.000000,0.000000,0.000000,0.0,0.000000,0.0,0.000000,0.000000,0.000000,1.000000,0.0,0.0,0.000000,1.0,0.0,0.0,0.000000,0.0,0.000000,0.000000
2,1.000000,1.000000,0.000000,0.0,0.000000,0.0,1.000000,0.000000,1.000000,0.000000,0.0,0.0,1.000000,1.0,0.0,0.0,0.000000,0.0,0.000000,1.000000
3,1.000000,0.000000,0.666667,0.0,1.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.000000,1.0,0.0,0.0,0.000000,0.0,0.000000,1.000000
4,0.000000,0.000000,0.000000,0.0,0.000000,0.0,0.000000,1.000000,0.000000,0.000000,0.0,0.0,0.000000,1.0,0.0,0.0,0.666667,0.0,0.000000,1.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,1.000000,1.000000,1.000000,1.0,1.000000,0.0,1.000000,1.000000,1.000000,0.000000,1.0,0.0,1.000000,0.0,1.0,1.0,0.000000,1.0,1.000000,1.000000
96,0.000000,1.000000,1.000000,1.0,1.000000,1.0,1.000000,0.000000,0.333333,0.000000,1.0,1.0,1.000000,0.0,1.0,1.0,1.000000,1.0,1.000000,1.000000
97,0.909091,0.909091,1.000000,1.0,1.000000,1.0,0.909091,0.000000,1.000000,0.000000,1.0,1.0,0.909091,0.0,1.0,1.0,1.000000,1.0,1.000000,1.000000
98,0.000000,1.000000,0.000000,1.0,0.000000,1.0,1.000000,0.000000,1.000000,0.000000,1.0,1.0,1.000000,0.0,1.0,1.0,0.000000,1.0,1.000000,1.000000


Next, we'll need to apply the cluster reducer down to k=1 and calculate the average distance between clusters at each step to get a sense of how many clusters should be included.

In [51]:
simulated_node_distances

array([[ 0.,  3.,  4., ..., 16., 14., 16.],
       [ 3.,  0.,  7., ..., 19., 15., 17.],
       [ 4.,  7.,  0., ..., 12., 10., 12.],
       ...,
       [16., 19., 12., ...,  0.,  4.,  2.],
       [14., 15., 10., ...,  4.,  0.,  6.],
       [16., 17., 12., ...,  2.,  6.,  0.]])

In [52]:
np.mean(simulated_node_distances, axis = 0)

array([ 9.1 ,  9.9 ,  9.08,  9.56, 10.26,  9.1 ,  9.1 ,  9.1 ,  8.7 ,
        9.1 ,  9.92,  8.72,  8.64,  9.1 ,  9.42,  9.1 ,  9.92,  8.74,
        9.9 ,  9.84,  9.9 ,  9.1 ,  8.7 ,  9.1 ,  9.14,  7.98,  8.78,
        8.06,  7.96,  8.34,  9.24,  8.32,  9.42,  8.46,  7.98,  7.98,
        7.98,  8.38,  8.32,  8.32,  8.78,  8.  ,  7.96,  7.98,  9.6 ,
        8.78,  7.98,  9.12,  8.36,  8.8 ,  8.16,  8.06,  8.12,  8.08,
        8.82,  8.5 ,  8.16,  8.52,  8.86,  9.34,  8.16,  8.94,  9.32,
        8.46,  8.14,  8.18,  9.34,  8.58,  8.42,  8.14,  8.52,  8.46,
        8.58,  8.8 ,  8.48,  9.28,  9.32,  9.28,  9.2 , 10.38,  9.28,
        9.28,  9.26,  9.32, 10.46,  9.6 ,  9.28,  9.26,  8.94,  9.62,
        9.28,  9.28,  9.5 ,  9.18,  9.22, 10.42,  9.3 ,  9.28,  8.48,
        9.2 ])

In [53]:
np.mean(np.mean(simulated_node_distances, axis = 0))

8.9116

**2F. and 2G.**  
So the average distance of each node to every other node is 8.4.  Ideally, this number gets larger as we decrease the number of clusters.  To that end, lets wrap all of these functions into a single function which stores the results at each stage as well as the average distances.

In [342]:
def binary_clusterer(node_dataframe_input, tolerance_input = 1, distance_tolerance_pct = 0):
    '''
    input: node_dataframe_input: a dataframe with nodes in the rows and binary features in the columns
           tolerance_input: value to be added to the minimum distance to be included in the closest nodes
           distance_tolerance_pct: for larger datasets, it may be reasonable to drop more than just the nearest nodes
               such that k is reduced more quickly in the beginning. This value is the percent to be added to the 
               min/max values used to calculate which nodes to drop
    output: clusters: for each step, the resulting hybrid clusters
            average_distances: for each step, the resulting average distance to every other cluster
            k: the number of centroids
    '''
    centroids = []
    average_distances = []
    ks = []
    node_dataframe = node_dataframe_input
      
    k = len(node_dataframe_input)
    
    while k>=1:
        # save results of current step
        centroids.append(node_dataframe)
        average_distances.append(np.mean(np.mean(node_distancer(node_dataframe), axis = 0)))
        ks.append(k)
        
        # apply the distance_tolerance_pct more heavily when k is closer to the maximum number of nodes,
            # then back off as we approach convergence by applying a weight relative to k
        graduated_tolerance_pct = k/len(node_dataframe_input) * distance_tolerance_pct
        
        # reduce node_dataframe
        node_dataframe = node_reducer(node_dataframe, tolerance_input, graduated_tolerance_pct)
        
        # set k
        k = len(node_dataframe)
    
    cluster_results = [centroids,average_distances,ks]
    
    # reverse the order (from smallest number of clusters to greatest) to make interpretation easier
    cluster_results[0].reverse()
    cluster_results[1].reverse()
    cluster_results[2].reverse()
    
    return(cluster_results)
        
    
    

Test with 0% tolerance (this should have the maximum number of clusters and take the longest to process)

In [343]:
cluster_results = binary_clusterer(simulated_games,1,0)

In [344]:
cluster_results_comparison = pd.DataFrame({'k_clusters':cluster_results[2],
                                          'average_cluster_distance':cluster_results[1]})

In [345]:
cluster_results_comparison.shape

(48, 2)

In [352]:
cluster_results_comparison

,k_clusters,average_cluster_distance
0,2,5.086420
1,3,5.650206
2,4,5.530093
3,5,6.302222
4,6,6.793210
5,7,7.217687
6,8,7.323717
7,9,7.383767
8,10,7.689846
9,11,7.925921


Lets compare with a 50% tolerance:

In [347]:
cluster_results_high_tolerance = binary_clusterer(simulated_games,1,0.50)

In [348]:
cluster_results_high_tolerance_comparison = pd.DataFrame({'k_clusters':cluster_results_high_tolerance[2],
                                                          'average_cluster_distance':cluster_results_high_tolerance[1]})

In [349]:
cluster_results_high_tolerance_comparison.shape

(15, 2)

In [353]:
cluster_results_high_tolerance_comparison

,k_clusters,average_cluster_distance
0,2,5.481481
1,3,6.148148
2,4,6.987500
3,6,7.244444
4,7,7.209524
5,9,7.396159
6,11,7.857668
7,13,7.662722
8,15,8.109926
9,20,8.432167


With a 50% tolerance, convergence is reached in only 15 iterations vs the 48 with 0% tolerance. This should lead to significant gains in performance in larger datasets.

Based on these results, it seems that 6 clusters is about where the 'elbow' is since the jump from 4.48 to 7.04 is large relative to the other marginal distance changes.  This being the case, we'll select the centroids for k=6.  We know that we intended for 4 clusters, but because we introduce randomness with how the clusters were created, 6 seems feasible.

In [329]:
cluster_centroids = cluster_results[0][3]

In [330]:
cluster_centroids

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19
0,0.0,0.666667,0.00,0.416667,0.5,1.0,0.388889,0.0,0.5,0.222222,0.75,1.0,0.5,0.250000,0.0,0.50,0.0,0.5,0.5,0.166667
2,0.0,1.000000,0.75,1.000000,1.0,1.0,0.000000,0.0,1.0,0.750000,0.00,1.0,1.0,1.000000,0.0,1.00,0.0,0.0,1.0,0.000000
3,0.0,1.000000,0.00,0.000000,1.0,1.0,0.000000,0.0,1.0,0.000000,0.00,1.0,1.0,0.000000,0.0,1.00,0.0,0.0,1.0,0.000000
4,1.0,1.000000,0.25,1.000000,1.0,1.0,1.000000,0.0,1.0,0.000000,1.00,1.0,1.0,0.000000,1.0,0.25,1.0,1.0,1.0,1.000000
5,1.0,1.000000,1.00,1.000000,1.0,0.0,1.000000,0.0,1.0,0.000000,1.00,1.0,1.0,0.666667,1.0,1.00,1.0,1.0,1.0,1.000000


Finally, now that we have our centroids, we can classify each of the original games into clusters by its nearest centroid:

In [331]:
def cluster_classifier(node_table_input, cluster_centroids):
    '''
    input: node_table_input: a dataframe with nodes in the rows and binary features in the columns
           cluster_centroids: the centroids resulting from the binary clustering algorithm
    output: a list which classifies each node into a cluster based on shortest distance
    
    '''
    clusters = []
    
    for i in range(len(node_table_input)):
        node_distances = []
        
        # get distance to each cluster centroid:
        for j in range(len(cluster_centroids)):
            node_distances.append(cluster_distance_calculator(node_table_input.iloc[i], cluster_centroids.iloc[j]))
        
        min_distance = min(node_distances)
        
        clusters.append(np.where(np.asarray(node_distances)==min_distance)[0][0])
    
    return(clusters)

In [332]:
node_clusters = cluster_classifier(simulated_games, cluster_centroids)

In [333]:
node_clusters[0:20]

[0, 0, 0, 0, 2, 0, 0, 0, 0, 0, 0, 2, 0, 0, 0, 0, 0, 0, 0, 0]

Just for fun, lets see how accurate our classifier if we select k=4 (the expected number of clusters):

In [334]:
# select index = 2, which is 4 clusters
cluster_centroids = cluster_results[0][2]
node_clusters = cluster_classifier(simulated_games, cluster_centroids)

We're expecting the groupings to show the first 25 in a cluster, the second 25 in a cluster, and so on, so lets investigate by each set of 25. First though, lets see how many nodes were classified in each cluster:

In [335]:
print(node_clusters.count(0))
print(node_clusters.count(1))
print(node_clusters.count(2))
print(node_clusters.count(3))

47
23
5
25


Looks like several from the first cluster were pulled into the second.  Lets see how it looks per expected cluster:

In [336]:
node_clusters[0:25]

[0, 0, 0, 0, 2, 0, 0, 0, 0, 0, 0, 2, 0, 0, 0, 0, 0, 0, 0, 0, 2, 0, 0, 0, 0]

Looks like this cluster was classified as '2' so lets see how many aren't 2:

In [337]:
len(np.where(np.asarray(node_clusters[0:25])!=2)[0])

22

84% isn't bad considering the noise we threw in.  Lets check out the other clusters:

In [338]:
node_clusters[25:50]

[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]

In [339]:
len(np.where(np.asarray(node_clusters[25:50])!=1)[0])

25

64% of the next 25 were in the same cluster.

In [340]:
node_clusters[50:75]

[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 1, 1, 1, 1, 2, 1, 1, 1, 1]

In [341]:
len(np.where(np.asarray(node_clusters[50:75])!=1)[0])

2

56% of the next 25 were in the same cluster.

In [199]:
node_clusters[75:100]

[3, 3, 1, 1, 1, 3, 3, 3, 1, 3, 3, 3, 3, 3, 3, 3, 3, 1, 3, 3, 3, 3, 3, 1, 3]

In [200]:
len(np.where(np.asarray(node_clusters[75:100])!=3)[0])

6

76% of the last 25 were in the same cluster.

Overall, we're showing between 56% and 84% accuracy in grouping the nodes into the same clusters as we originally created. However, the inclusion of 25% noise when generating the data in an effort to appear more realistic is obfuscating the clarity of the results.  This being the case, we'll test again with only a 10% difference between any node and its medoid:

In [235]:
first_medoid_cluster_simple = cluster_creator(first_medoid, [0,0.1], 24)
second_medoid_cluster_simple = cluster_creator(second_medoid, [0,0.1], 24)
third_medoid_cluster_simple = cluster_creator(third_medoid, [0,0.1], 24)
fourth_medoid_cluster_simple = cluster_creator(fourth_medoid, [0,0.1], 24)

In [236]:
simulated_games_simple = [first_medoid] + first_medoid_cluster_simple + [second_medoid] + \
    second_medoid_cluster_simple + [third_medoid] + third_medoid_cluster_simple + [fourth_medoid] + \
    fourth_medoid_cluster_simple

In [237]:
simulated_games_simple = pd.DataFrame(np.array(simulated_games_simple))

In [238]:
cluster_results_simple = binary_clusterer(simulated_games_simple, 1)

In [240]:
cluster_results_simple_comparison = pd.DataFrame({'k_clusters':cluster_results_simple[2],
                                                  'average_cluster_distance':cluster_results_simple[1]})

In [242]:
cluster_results_simple_comparison[0:10]

,k_clusters,average_cluster_distance
0,2,5.500000
1,3,5.629630
2,4,5.303819
3,5,5.404444
4,6,5.833333
5,7,6.394558
6,8,6.679167
7,9,6.962963
8,10,6.820000
9,11,6.670893


In [245]:
cluster_centroids_simple = cluster_results_simple[0][2]

In [246]:
node_clusters_simple = cluster_classifier(simulated_games_simple, cluster_centroids_simple)

In [248]:
print(node_clusters_simple.count(0))
print(node_clusters_simple.count(1))
print(node_clusters_simple.count(2))
print(node_clusters_simple.count(3))

26
24
25
25


In [249]:
node_clusters_simple[0:25]

[2, 2, 1, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2]

In [251]:
len(np.where(np.asarray(node_clusters_simple[0:25])!=2)[0])

1

That's 96% (24/25) accurate for the first cluster!

In [252]:
node_clusters_simple[25:50]

[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]

In [255]:
len(np.where(np.asarray(node_clusters_simple[25:50])!=0)[0])

0

100% (25/25) accurate for the second cluster!

In [256]:
node_clusters_simple[50:75]

[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 1]

In [258]:
len(np.where(np.asarray(node_clusters_simple[50:75])!=1)[0])

2

92% (23/25) accurate for the third cluster.

In [260]:
node_clusters_simple[75:100]

[3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3]

In [261]:
len(np.where(np.asarray(node_clusters_simple[75:100])!=3)[0])

0

100% (25/25) accurate for the third cluster!

It looks like our classifier is at 97% accurate when we reduce the noise around the medoids to 10% from 25%.  This isn't surprising, but the elbow method of identifying the right number of clusters didn't exactly make choosing the right number of clusters obvious.  In fact, at k=4 clusters, we're seeing a minimum of the average distance between clusters.  This leads me to believe that I've misinterpreted the average_cluster_distance and I should, instead, be attempting to select the k for which the average_cluster_distance is small and the k+1 distance is a large jump.

___
# Takeaways

In this script, an unsupervised binary classification algorithm was devised and tested with simulated data centered around 4 medoids. At 25% noise added around the 4 medoids, we had a 70% accuracy in classification.  At 10%, however, we saw a 97% accuracy.  

Additionally, we learned that the correct approach to selecting k clusters is to identify the k for which the average distance between clusters is small and the k+1 cluster sees a relatively large increase in average distance.